In [1]:
import pandas as pd
import csv
import time

In [2]:
def rearrange_csv_columns(path):
    new_path = '/'.join(path.split('/')[:-1]) + '/new_' + path.split('/')[-1]
    num_of_cols = 35
    prob_col =  11
    prob_string_list = [' Jr."' , ' Jr.']
     
    with open(path, "r") as f_in, open(new_path, "w") as f_out:
        reader = csv.reader(f_in)
        writer = csv.writer(f_out)
        for row in reader:
            if len(row) == num_of_cols + 1:
                assert row[prob_col] in prob_string_list, row[prob_col] + ' -> id:' + str(row[0])
                new_row[:prob_col-1] = row[:prob_col-1]
                new_row[prob_col-1] = row[prob_col-1] + row[prob_col]
                new_row[prob_col:] = row[prob_col+1:]
            else:
                new_row = row

            writer.writerow(new_row)

    f_in.close()
    f_out.close()
    
    return new_path

In [86]:
# subjects 11, 22 are missing => 21 subjects in total
subjects_dict = dict()
with open('data/policy_agenda.csv', "r") as f_in:
    reader = csv.reader(f_in)
    for row in reader:
        subjects_dict[float(row[0])] = row[1]
    f_in.close()

##### outer bills file

In [4]:
_ = rearrange_csv_columns('data/bills/bills.csv')
bills = pd.read_csv('data/bills/new_bills.csv')

/home/foivos/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,17,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(bills.shape)
print(bills.columns)
bills.Cong.unique()

(279198, 35)
Index(['idNEW', 'idOLD', 'BillNum', 'BillType', 'Cong', 'IntrDate',
       'ShortTitle', 'OfficialTitle', 'PopTitle', 'SpThomasID', 'SpName',
       'SpState', 'SpDist', 'UpdatedAt', 'CoSpThID', 'PLawNo', 'MinorBill',
       'PLawNoFull', 'compLaw', 'Major', 'Minor', 'ChRef', 'RankRef', 'MemRef',
       'SubChRef', 'SubRankRef', 'ImpBill', 'URL', 'isBill', 'Majority',
       'Senate', 'LeadRef', 'SpParty', 'commRefs', 'timestamp'],
      dtype='object')


array([ 93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113])

##### inner bills file

In [32]:
_ = rearrange_csv_columns('data/bills/data/congress/output/bills.csv')
bills = pd.read_csv('data/bills/data/congress/output/new_bills.csv')

/home/foivos/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,17,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
print(bills.shape)
print(bills.columns)
bills.Cong.unique()

(292868, 35)
Index(['idNEW', 'idOLD', 'BillNum', 'BillType', 'Cong', 'IntrDate',
       'ShortTitle', 'OfficialTitle', 'PopTitle', 'SpThomasID', 'SpName',
       'SpState', 'SpDist', 'UpdatedAt', 'CoSpThID', 'PLawNo', 'MinorBill',
       'PLawNoFull', 'compLaw', 'Major', 'Minor', 'ChRef', 'RankRef', 'MemRef',
       'SubChRef', 'SubRankRef', 'ImpBill', 'URL', 'isBill', 'Majority',
       'Senate', 'LeadRef', 'SpParty', 'commRefs', 'timestamp'],
      dtype='object')


array([ 93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114])

In [34]:
BILL_COLUMNS = ['idNEW', 'Cong', 'SpParty', 'Major', 'SpState']
bills = bills[BILL_COLUMNS]

In [35]:
print(bills.shape)
print(bills.columns)
bills.head(1)

(292868, 5)
Index(['idNEW', 'Cong', 'SpParty', 'Major', 'SpState'], dtype='object')


,idNEW,Cong,SpParty,Major,SpState
0,1,93,D,20.0,IL


In [88]:
bills = bills[(bills['Cong'] == 114) | (bills['Cong'] == 113)]

### Grouping

In [89]:
grouped_bills = bills.groupby(BILL_COLUMNS[1:]).size()

In [96]:
with open('data/grouped_bills.csv', "w") as f_out:
    writer = csv.writer(f_out)    
    writer.writerow(['congress', 'party', 'major', 'state', 'count'])
    for index, value in grouped_bills.iteritems():
        writer.writerow([index[0], index[1], index[2], index[3], value])        
    f_out.close()